In [2]:
import numpy as np

class Oracle:
    '''Provides an interface for evaluating a function and its derivative at arbitrary point'''
    
    def value(self, x: np.ndarray) -> float:
        '''Evaluates the underlying function at point `x`

        Args:
            x: a point to evaluate funciton at

        Returns:
            Function value
        '''
        raise NotImplementedError()
        
    def gradient(self, x: np.ndarray) -> np.ndarray:
        '''Evaluates the underlying function derivative at point `x`

        Args:
            x: a point to evaluate derivative at

        Returns:
            Function derivative
        '''
        raise NotImplementedError()

In [41]:
import numpy as np

class AdaGrad:
    '''Класс для реализации адаптивного градиентного спуска

    Значения:
        eta: скорость обучения
        epsilon: сглаживающий коэффициент
    '''

    eta: float
    epsilon: float

    def __init__(self, *, eta: float = 0.1, epsilon: float = 1e-8):

        self.eta = eta
        self.epsilon = epsilon
        self.vector = 0 # вектор накопленных градиентов

    def optimize(self, oracle: Oracle, x0: np.ndarray, *,
                 max_iter: int = 100, eps: float = 1e-5) -> np.ndarray:
        '''Оптимизирует функцию со стартовой точки х0.
        Оптимизация заканчивается когда мы достигли макс. числа итераций
        или если наша точка оказалась ниже порога

        Аргументы функции:
            oracle: функция которую нужно оптимизировать
            x0: стартовая точка спуска
            max_iter: максимальное число итераций (ограничение)
            eps: граница

        Возвращает:
            Точку остановки
        '''

        x = x0
        self.vector = np.zeros_like(x0)

        for _ in range(max_iter):
            gradient = oracle.gradient(x) # вычисление градиента в текущей точке
            if np.linalg.norm(gradient) < eps:
                # проверка на пересечение порога
                break
            self.vector += np.power(gradient, 2)
            a = (self.eta / (np.sqrt(self.vector + self.epsilon))) * gradient
            x -= a
        
        return x

In [42]:
# Example usage of Oracle and RMSProp
class QuadraticOracle(Oracle):
    '''Example quadratic function for optimization'''

    def value(self, x: np.ndarray) -> float:
        return np.sum(x ** 2)  # Function value f(x) = x^2
    
    def gradient(self, x: np.ndarray) -> np.ndarray:
        return 2 * x  # Gradient f'(x) = 2x


# Using RMSProp to optimize a quadratic function
oracle = QuadraticOracle()
initial_point = np.array([10.0, -5.0])  # Initial point
optimizer = AdaGrad(eta=0.1)

optimal_point = optimizer.optimize(oracle, initial_point)
print("Оптимальная точка:", optimal_point)


Оптимальная точка: [ 8.2083308  -3.28320738]
